# Data preprocessing
it contains two large modules:
process X and process Y

In [1]:
# import arcpy
import numpy as np
import os, glob
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
arcpy.CheckExtension('ImageAnalyst')

'Available'

## process X
### X from sentinal 2 L2A data
#### check bands and resolutions, resample all bands to 10m
resolution 10m: band2(blue), band3(green), band4(red), band8(nir)<br>
need to be resampled to R10m: band5, band6, band7, band8A, band11, band12<br>
band1, band9, band10 are useless, only for correction
so the total num of bands is 10<br>
followed by ASTER band4-9 and Geo-physical data

#### calculate NDVI
set threshold as 0.3, band3 is red, band7 is NIR

#### layer number and rock type
| Layer | Rock |
|-------|------|
| 0     | Vegetation|
| 1     | Unkown Rocks|
| 2     | Carbonate_sediment|
| 3     | Dolerite|
| 4     | Feldspathic_sediment|
| 5     | Felsic_volcanic|
| 6     | Gneiss|
| 7     | Granite|
| 8     | Mafic_volcanic|
| 9     | Quartz_sediment|

## Export training data X, Y with tile of 256 x 256

#### split X and Y into tiles

In [5]:
from src.DeepRock.utils import split2Tiles

In [11]:
# normalize input remote sensing data, and save as numpy array
block_id = 4
# read the raster block
raster = arcpy.Raster('C:/Users/yuch/Documents/ArcGIS/Projects/mount_isa/BLOCK{0}.tif'.format(block_id))
raster_num = arcpy.RasterToNumPyArray(raster)
# normalize each band by (x-min)/(max-min)
for i in range(raster_num.shape[0]):
    raster_num[i,:,:] = (raster_num[i,:,:] - raster_num[i,:,:].min()) / (raster_num[i,:,:].max() - raster_num[i,:,:].min())
    
np.save('./data/Blocks/block{0}'.format(block_id), raster_num)

In [15]:
# save labels as numpy array
block_id = 4
# read the raster block
raster = arcpy.Raster('C:/Users/yuch/Documents/ArcGIS/Projects/mount_isa/Labels_BLOCK{0}.tif'.format(block_id))
raster_num = arcpy.RasterToNumPyArray(raster)
    
np.save('./data/Blocks/labels_block{0}'.format(block_id), raster_num)

In [23]:
# use block 1, 3, 4 as train data
# and block 2 as test data
split2Tiles('./data/test/Y/', np.load('./data/Blocks/labels_block2.npy'), suffix='block2')

split into 294 tiles


#### remove tile with bad boundary

In [2]:
from src.DeepRock.utils import remove_boundary

In [3]:
remove_boundary('./data/train/')
remove_boundary('./data/test/')

removing tiles with bad boundary
removing tiles with bad boundary
